In [1]:
from pprint import pprint

In [41]:
import fitz  # import package PyMuPDF

# Open some document, for example a PDF (could also be EPUB, XPS, etc.)
doc = fitz.open("Reports/tcs.pdf")

In [3]:
import requests
from io import BytesIO

def open_pdf_from_url(pdf_url):
    try:
        # Download PDF content from the URL
        response = requests.get(pdf_url)
        response.raise_for_status()

        # Open the PDF using PyMuPDF
        pdf_document = fitz.open(stream=BytesIO(response.content))
        
        # Replace the following line with your PDF manipulation code
        # For example, you can iterate through pages and extract text, etc.
        for page_number in range(pdf_document.page_count):
            page = pdf_document[page_number]
            text = page.get_text()
            print(f"Page {page_number + 1} text:\n{text}")

    except Exception as e:
        print(f"Error: {e}")

In [4]:
page = doc[6]  # this is the first page

In [5]:
blocks = page.get_text("dict")["blocks"]
for b in blocks:  # iterate through the text blocks
    for l in b["lines"]:  # iterate through the text lines
        for s in l["spans"]:  # iterate through the text spans
            if s["flags"] == 20:  # 20 targets bold
                print(s['text'])

In [6]:
import fitz
import re

In [7]:
doc = fitz.open("Reports/tcs.pdf")

In [9]:
text = ""
for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text += page.get_text("text")
text=re.sub(r'\n', ' ', text)

In [10]:
tcspage=doc[45]

In [11]:
blocks = tcspage.get_text("dict")["blocks"]

In [13]:
tabs = tcspage.find_tables()
l=len(tabs.tables)
page_all_table_bbox=[]
for i in range(l):
    page_all_table_bbox.append(tabs[i].bbox)

In [14]:
tabs[0].extract()

[['Transactoi ns',
  'Tata Sons\nPrivate\nLimited',
  'Other than\nwholly owned\nsubsidiaries of\nthe Company',
  'Subsidiaries of Tata\nSons Private Limited',
  'Associates/joint\nventures of Tata Sons\nPrivate Limited and their\nsubsidiaries',
  'Total'],
 ['IT/ITE services rendered', '38', '1,063', '1,152', '2,506', '4,759'],
 ['Other income', '-', '-', '-', '-', '-'],
 ['Procurement of goods and\nservices', '-', '59', '577', '363', '1,000'],
 ['Brand equity contribution', '99', '-', '-', '-', '99'],
 ['Non IT/ITE services availed', '1', '-', '23', '59', '83'],
 ['Lease rental', '-', '-', '56', '47', '103']]

In [15]:
page_all_table_bbox

[(54.125, 192.6009286733774, 539.875, 325.8509063720703),
 (54.125, 348.3950453538161, 539.875, 481.6450500488281),
 (54.125, 498.1891784667969, 539.875, 629.1242065429688)]

In [16]:
text=""
table_counter=0
for b in blocks:  # iterate through the text blocks
   if "lines" in b: 
    for l in b["lines"]:  # iterate through the text lines
        for s in l["spans"]:  # iterate through the text spans
            if is_rectangle_outside_all(b["bbox"], page_all_table_bbox):
                text += ' ' + s["text"]
            else:
                text+= ' ' + f'{tabs[table_counter].extract()}'
                table_counter=1


In [17]:
tcspage.get_textbox(rect=(54.125, 192.6009286733774, 539.875, 325.8509063720703))

'Transactions\nTata Sons \nPrivate \nLimited\nOther than \nwholly owned \nsubsidiaries of \nthe Company\nSubsidiaries of Tata \nSons Private Limited\nAssociates/joint \nventures of Tata Sons \nPrivate Limited and their \nsubsidiaries\nTotal\nIT/ITE services rendered\n38\n1,063\n1,152\n2,506\n4,759\nOther income\n-\n-\n-\n-\n-\nProcurement of goods and \nservices\n-\n59\n577\n363\n1,000\nBrand equity contribution\n99\n-\n-\n-\n99\nNon IT/ITE services availed\n1\n-\n23\n59\n83\nLease rental\n-\n-\n56\n47\n103'

In [18]:
print(text)

 Notice   |  46 Integrated Annual Report 2022-23 Item No. 6 Pursuant to the amended Regulation 23 of SEBI (Listing Obligations and Disclosure Requirements) Regulations, 2015 (‘‘SEBI Listing  Regulations’’), the threshold limit for determination of material Related Party Transactions is the lower of ₹1,000 crores   (Rupees One thousand crores) or 10% (ten percent) of the annual consolidated turnover of the listed entity as per the last audited  financial statements of the listed entity and such material related party transactions exceeding the limits, would require prior approval  of Members by means of an ordinary resolution. Tata Consultancy Services Limited (“the Company” or “TCS”), being a globally recognised provider of IT services, participates in the  digitisation initiatives of entities within Tata group and partners in respective entities’ growth and transformation journeys. During the  course of rendering such services, the Company also leverages niche skills, capabilities and

In [45]:
chunks =[]
flag = "" #heading or normal
i=0
for b in blocks:  # iterate through the text blocks
   if "lines" in b: 
    for l in b["lines"]:  # iterate through the text lines
        for s in l["spans"]:  # iterate through the text spans
            if ("Bold" in s["font"] or "DemiB" in s["font"]) and len(l["spans"])==1:
            #  if '\n' in s["text"]:
                chunks.append(s["text"])
                if flag == "normal":
                    i=i+1
                    flag = "heading"
                # print(s['text'], s['size'] , s["font"])
            else:
                if flag == "":
                    chunks.append(s["text"])
                chunks[i] = chunks[i] +" " + s["text"]
                flag = "normal"

In [88]:
tabs=tcspage.find_tables()

In [152]:
tabs[0].to_pandas()

,"Year ended March 31, 2023",Col1,Col2,Col3,Col4,(` crore)
0,Transactoi ns,Tata Sons\nPrivate\nLimited,Other than\nwholly owned\nsubsidiaries of\nthe...,Subsidiaries of Tata\nSons Private Limited,Associates/joint\nventures of Tata Sons\nPriva...,Total
1,IT/ITE services rendered,38,"1,063","1,152","2,506","4,759"
2,Other income,-,-,-,-,-
3,Procurement of goods and\nservices,-,59,577,363,"1,000"
4,Brand equity contribution,99,-,-,-,99
5,Non IT/ITE services availed,1,-,23,59,83
6,Lease rental,-,-,56,47,103


In [89]:
tcspage.get_textbox(rect=(54.125, 192.6009286733774, 539.875, 325.8509063720703))

'Transactions\nTata Sons \nPrivate \nLimited\nOther than \nwholly owned \nsubsidiaries of \nthe Company\nSubsidiaries of Tata \nSons Private Limited\nAssociates/joint \nventures of Tata Sons \nPrivate Limited and their \nsubsidiaries\nTotal\nIT/ITE services rendered\n38\n1,063\n1,152\n2,506\n4,759\nOther income\n-\n-\n-\n-\n-\nProcurement of goods and \nservices\n-\n59\n577\n363\n1,000\nBrand equity contribution\n99\n-\n-\n-\n99\nNon IT/ITE services availed\n1\n-\n23\n59\n83\nLease rental\n-\n-\n56\n47\n103'

In [90]:
tabs[2].bbox

(54.125, 498.1891784667969, 539.875, 629.1242065429688)

In [98]:

# Example usage:
all_rectangles = [
    (54.125, 192.6009286733774, 539.875, 325.8509063720703),
    (54.125, 348.3950453538161, 539.875, 481.6450500488281),
    (54.125, 498.1891784667969, 539.875, 629.1242065429688)
]

input_rectangle = (54.0, 696.6140747070312, 540.0008544921875, 703.6140747070312)

result = is_rectangle_outside_all(input_rectangle, all_rectangles)
print(result)  # True (input rectangle is outside of all rectangles)


True


In [96]:
tcspage.get_textbox(rect=(54.0, 696.6140747070312, 540.0008544921875, 703.6140747070312))

'Notice   |  46\nIntegrated Annual Report 2022-23'

In [ ]:
pprint(blocks)

In [20]:
tcspage.get_textbox(rect=(54.0, 696.6140747070312, 540.0008544921875, 703.6140747070312))

'Notice   |  46\nIntegrated Annual Report 2022-23'

In [42]:
docchunks =[]
flag = "" #heading or normal
i=0
for page_num in range(doc.page_count): 
 page = doc.load_page(page_num)
 blocks = page.get_text('dict')["blocks"]
 for b in blocks:  # iterate through the text blocks
   if "lines" in b: 
    for l in b["lines"]:  # iterate through the text lines
        for s in l["spans"]:  # iterate through the text spans
            if ("Bold" in s["font"] or "DemiB" in s["font"]) and len(l["spans"])==1:
            #  if '\n' in s["text"]:
                docchunks.append(s["text"])
                if flag == "normal":
                    i=i+1
                    flag = "heading"
                # print(s['text'], s['size'] , s["font"])
            else:
                if flag == "":
                    docchunks.append(s["text"])
                docchunks[i] = docchunks[i] +" " + s["text"]
                flag = "normal"

In [43]:
len(docchunks)

7544

In [44]:
docchunks

['Innovate, Adapt, Thrive Innovate, Adapt, Thrive',
 'Integrated Annual Report About  TCS Tata Consultancy Services is an IT services, consulting and  business solutions organization that has been partnering with  many of the world’s largest businesses in their transformation  journeys for over 55 years. Its consulting-led, cognitive powered,  portfolio of business, technology and engineering services and  solutions is delivered through its unique Location Independent  Agile™ delivery model, recognized as a benchmark of excellence  in software development. A part of the Tata group, India’s largest multinational  business group, TCS has over 614,000 of the world’s best- trained consultants in 55 countries. The company generated  consolidated revenues of US $27.9 billion in the fiscal year  ended March 31, 2023, and is listed on the BSE and the NSE in  India.  TCS’ proactive stance on climate change and award-winning  work with communities across the world have earned it a  place in lead

In [74]:
len(chunks)

7544